In [ ]:
!pip install transformers
!pip install Unidecode
!pip install datasets evaluate accelerate
!pip install sacrebleu jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/DL_Final/dataset.zip"
data_path = "corpus-title.txt"

unzip:  cannot find or open /content/drive/MyDrive/DL_Final/dataset.zip, /content/drive/MyDrive/DL_Final/dataset.zip.zip or /content/drive/MyDrive/DL_Final/dataset.zip.ZIP.


In [ ]:
import numpy as np
import unidecode
import re
import random
import datasets

In [ ]:
accented_chars_vietnamese = [
    'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ',
    'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ',
    'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự',
    'í', 'ì', 'ỉ', 'ĩ', 'ị',
    'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ',
    'đ',
]

accented_chars_vietnamese.extend([c.upper() for c in accented_chars_vietnamese])
alphabet = list("aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789! \"#$%&''()*+,-./:;<=>?@[\]^_{|}~")

chars_regrex = '[aàảãáạăằẳẵắặâầẩẫấậAÀẢÃÁẠĂẰẲẴẮẶÂẦẨẪẤẬoòỏõóọôồổỗốộơờởỡớợOÒỎÕÓỌÔỒỔỖỐỘƠỜỞỠỚỢeèẻẽéẹêềểễếệEÈẺẼÉẸÊỀỂỄẾỆuùủũúụưừửữứựUÙỦŨÚỤƯỪỬỮỨỰiìỉĩíịIÌỈĨÍỊyỳỷỹýỵYỲỶỸÝỴnNvVmMCG]'
same_chars = {
    'a': ['á', 'à', 'ả', 'ã', 'ạ', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ'],
    'A': ['Á','À','Ả','Ã','Ạ','Ấ','Ầ','Ẩ','Ẫ','Ậ','Ắ','Ằ','Ẳ','Ẵ','Ặ'],
    'O': ['Ó','Ò','Ỏ','Õ','Ọ','Ô','Ố','Ồ','Ổ','Ỗ','Ộ','Ơ','Ớ','Ờ','Ở','Ỡ','Ợ','Q'],
    'o': ['ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ','ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'q'],
    'e': ['é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ê'],
    'E': ['É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ê'],
    'u': ['ú', 'ù', 'ủ', 'ũ', 'ụ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ư'],
    'U': ['Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Ư'],
    'i': ['í', 'ì', 'ỉ', 'ĩ', 'ị'],
    'I': ['Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị'],
    'y': ['ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'v'],
    'Y': ['Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'V'],
    'n': ['m'],
    'N': ['N'],
    'v': ['y'],
    'V': ['Y'],
    'm': ['n'],
    'M': ['N'],
    'C': ['G'],
    'G': ['C']
}

indices = [alphabet.index(c) + 4 for c in accented_chars_vietnamese]
vocab_size = len(alphabet)
litle_char = list('"#$%&''()*+,-./:;<=>?@[\]^_{|}~')

In [ ]:
def remove_random_accent(text, ratio=0.8):
        text = ''.join(text)
        words = text.split()
        mask = np.random.random(size=len(words)) < ratio

        for i in range(len(words)):
            if mask[i]:
                words[i] = unidecode.unidecode(words[i])
                break

        return ' '.join(words)

def remove_random_space(text):
        text = ''.join(text)
        words = text.split()
        n_words = len(words)
        start = np.random.randint(low=0, high=n_words, size=1)[0]

        if start + 3 < n_words:
            end = np.random.randint(low=start, high=start + 3, size=1)[0]
        else:
            end = np.random.randint(low=start, high=n_words, size=1)[0]

        out = ' '.join(words[:start])  + ' ' + ''.join(words[start:end + 1]) + ' ' + ' '.join(words[end + 1:])

        return out.strip()

def _char_regrex(text):
        match_chars = re.findall(chars_regrex, text)

        return match_chars

def _random_replace(text, match_chars):
        replace_char = match_chars[np.random.randint(low=0, high=len(match_chars), size=1)[0]]
        insert_chars = same_chars[unidecode.unidecode(replace_char)]
        insert_char = insert_chars[np.random.randint(low=0, high=len(insert_chars), size=1)[0]]
        text = text.replace(replace_char, insert_char, 1)

        return text

def change(text):
        match_chars = _char_regrex(text)
        if len(match_chars) == 0:
            return text
        text = ''.join(text)
        text = _random_replace(text, match_chars)

        return text

def replace_accent_chars(text, ratio=0.8):
        text = ''.join(text)
        words = text.split()
        mask = np.random.random(size=len(words)) < ratio

        for i in range(len(words)):
            if mask[i]:
                words[i] = change(words[i])
                break

        return ' '.join(words)
def load_data(data_path):
        # Load data from the file
        with open(data_path, 'r', encoding='utf-8') as file:
            #lines = [line.strip().split('\t') for line in file.readlines()]
            lines = [file.readline().strip() for _ in range(3000)]
            print("Reading dataset with size: " + str(len(lines)))
        return lines

In [ ]:
import csv

def generate_csv(csv_path, target,input):
    with open(csv_path, 'w', newline='',encoding='utf-8') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for i in range(len(target)):
     	    # Adding the task's prefix to input
            input_text = input[i]
            writter.writerow([input_text, target[i]])


dataset = load_data(data_path)
target = []
input = []
function_list = [remove_random_accent, replace_accent_chars,remove_random_space,change]
random_function = random.choice(function_list)
random.shuffle(dataset)
for i in dataset:
    target.append(i)
    if random.random() < 0.8:
       k = random_function(i)
       input.append(k)
    else:
        input.append(i)
generate_csv("dataset.csv",target,input)


Reading dataset with size: 3000


In [ ]:
import csv

def generate_csv(csv_path, target,input):
    with open(csv_path, 'w', newline='',encoding='utf-8') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for i in range(len(target)):
     	    # Adding the task's prefix to input
            input_text = input[i]
            writter.writerow([input_text, target[i]])


dataset = load_data(data_path)
target = []
input = []
function_list = [remove_random_accent, replace_accent_chars,remove_random_space,change]
random_function = random.choice(function_list)
random.shuffle(dataset)
for i in dataset:
    target.append(i)
    if random.random() < 0.3:
       k = random_function(i)
       input.append(k)
    else:
        input.append(i)
generate_csv("evalset.csv",target,input)


Reading dataset with size: 3000


In [ ]:
import pandas as pd
df = pd.read_csv("dataset.csv")
df_eval = pd.read_csv("evalset.csv")
df.head()


,input,target
0,"Neu cho làm lại, liệu Ukraine có giữ được Crim...","Nếu cho làm lại, liệu Ukraine có giữ được Crim..."
1,Thi trấn Ollolai ở Italia bán nhà với giá hơn ...,Thị trấn Ollolai ở Italia bán nhà với giá hơn ...
2,"Cháu đòi tiền cơm, dì đòi tiền nhà.","Cháu đòi tiền cơm, dì đòi tiền nhà."
3,Nhan dân Sóc Trăng đón mừng sinh nhật Bác Hồ.,Nhân dân Sóc Trăng đón mừng sinh nhật Bác Hồ.
4,Tai nạn liên hoàn gây ách tắc giao thông trên ...,Tai nạn liên hoàn gây ách tắc giao thông trên ...


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
#BARTpho-syllable
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")





/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    # Tokenize the text and apply truncation
    return tokenizer(examples["input"],
                     text_target=examples["target"],
                     max_length=512,
                     truncation=True,
                     )

# Apply tokenization in a batched manner for efficiency
tokenized_datasets = datasets.Dataset.from_dict(df).map(
    preprocess_function,
    batched=True,
    remove_columns=datasets.Dataset.from_dict(df).column_names,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# Apply tokenization in a batched manner for efficiency
tokenized_datasets_eval = datasets.Dataset.from_dict(df_eval).map(
    preprocess_function,
    batched=True,
    remove_columns=datasets.Dataset.from_dict(df_eval).column_names,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
#Run if you want to re-train
#from transformers import AutoModelForSeq2SeqLM

#model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-syllable")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/DL_Final/checkpoint-4000")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
predictions = [
    "Nếu làm được như vậy thì chắc chắn sẽ khôTng còn trường nà tùy tiện tu tiềncaogây sự lo hắng của phụ huynh và ai khÔng có tiền thì kông cần dong"
]
references = [
    [
        "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 39.85576664202889,
 'counts': [24, 17, 11, 8],
 'totals': [32, 31, 30, 29],
 'precisions': [75.0,
  54.83870967741935,
  36.666666666666664,
  27.586206896551722],
 'bp': 0.8824969025845955,
 'sys_len': 32,
 'ref_len': 36}

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"sacrebleu": result["score"]}

In [ ]:

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    do_train=True,
    output_dir='/content/drive/MyDrive/DL_Final',
    num_train_epochs=50,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=20_000,
    save_strategy="steps",
    logging_steps=20_000,
    save_total_limit=5,
    predict_with_generate=True,
)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40030, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40030, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): 

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



In [ ]:
#trainer.train()
#trainer.save_model("/content/drive/MyDrive/DL_Final")
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.019208548590540886,
 'eval_sacrebleu': 87.61273359387789,
 'eval_runtime': 156.1705,
 'eval_samples_per_second': 19.21,
 'eval_steps_per_second': 2.401}

In [ ]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="/content/drive/MyDrive/DL_Final/checkpoint-4000")

In [ ]:
MAX_LENGTH = 512

# Define the text samples
texts = [
    "Toi phan bội chồng để rồi có tha voi người tình và bị phụ bạc.",
    "Bau Kiên đang ở tù thưởng tiền cho U23 Việt Nam, có sai không?.",
   "Hướng dan chi tiết phóng sinh cá chép tiễn Táo quan đúng cách.",
    "Cổ phieu SeABank do Mobifone chào bán với giá re hút nhà đầu tư.",
    "Giam giá vé trạm BOT Cần Thơ - Phụng Hiệp.",
    "Neu cho làm lại, liệu Ukraine có giữ được Crimea trước Nga?.",
    "Kiem tra vật tư nông nghiệp theo thông tư 45/2014/TT-BNNPTNT.",
    "Hoi âm bài viết 'Huyện Cẩm Thủy, Thanh Hóa: Nhiều hộ dân mòn mỏi chờ sổ đỏ' Đề xuất tỉnh cấp giấy chứng nhận quyền sử dụng đất.",
    "Nhan dân Sóc Trăng đón mừng sinh nhật Bác Hồ."



]

# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

# Print predictions
for text, pred in zip(texts, predictions):
    print("- " + pred['generated_text'])

- Toi phản bội chồng để rồi có tha voi người tình và bị phụ bạc.
- Bùi Kiên đang ở tù thưởng tiền cho U23 Việt Nam, có sai không?.
- Hướng dẫn chi tiết phóng sinh cá chép tiễn Táo quan đúng cách.
- Cổ phiếu SeABank do Mobifone chào bán với giá re hút nhà đầu tư.
- Giam giá vé trạm BOT Cần Thơ - Phụng Hiệp.
- Nếu cho làm lại, liệu Ukraine có giữ được Crimea trước Nga?.
- Kiểm tra vật tư nông nghiệp theo thông tư 45/2014/TT-BNNPTNT.
- Hồi âm bài viết 'Huyện Cẩm, Thanh: Nhiều hộ dân mòn mỏi chờ sổ đỏ' Đề xuất tỉnh cấp giấy chứng nhận quyền sử dụng đất.
- Người dân Sóc Trăng đón mừng sinh nhật Bác Hồ.
